Often when I browse different publically available datasets on for example [Kaggle](https://www.kaggle.com/) I find most of them are already [tidy](https://garrettgman.github.io/tidying/). Sure, to use the data for machine learning you must do feature engineering, but most of the data cleaning is already done. I believe that this bad, since most of the data in the "real" world is messy. 
<!-- TEASER_END -->
So, in this post, I will use [pandas](https://pandas.pydata.org/) to clean messy data from the [IMF data](https://www.imf.org/en/Data) homepage (International Financial Statistics) with the ultimate goal of producing a tidy dataframe.

I have downloaded 8 excel files and they contain the following data:

* Financial Market Prices, Equities, End of Period, Index
* Prices, Consumer Price Index, All items, Index
* Prices, Producer Price Index, All Commodities, Index
* Economic Activity, Industrial Production, Index
* Labor Force, Persons, Number of
* Unemployment, Persons, Number of
* Employment, Persons, Number of
* Labor Markets, Unemployment Rate, Percent

The post consists of the following four parts:
1. Setup and the first glance at a file
2. Parse and clean all files
3. Merge with external data
4. Save data

## 1. Setup and a first look at one file
First I import the libraries I need. Beyond pandas, I will use [pathlib](https://docs.python.org/3/library/pathlib.html), which is in the standard library, and the third party library [neat_panda](https://github.com/htp84/neat_panda) to clean the data. 

I also create a custom print function to add newlines, write in bold and colorize the printed text. A good explanation on how to modify the appearance of printed text in jupyter can be found [here](https://stackoverflow.com/questions/23271575/printing-bold-colored-etc-text-in-ipython-qtconsole).

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from neat_panda import spread, gather

In [2]:
def print2(value):
    print(f"\n\n\n\x1b[1;30;47m{value}\x1b[0m", end="\n\n")

To see which excel files I have in the current directory I use the *Path* function from *pathlib* and its *glob* method. Since *glob* returns a generator I use the *list* function to transform it to a list called **files**.

The files are named poorly, as seen below. However, I know there is information in the files that will help to identify them. Therefore, I parse the first file in **files**. I view its first 10 rows using the *head* method.

In [3]:
path = Path("../../imf/")

files = list(path.glob("*.xlsx"))
print2("Files:")
files

print2("Data from the first file:")
pd.read_excel(files[0]).head(10)




Files:



[PosixPath('../../imf/Prices_Production_and_Labor (1).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor (2).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor (3).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor (4).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor (5).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor (6).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor (7).xlsx'),
 PosixPath('../../imf/Prices_Production_and_Labor.xlsx')]




Data from the first file:



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105
0,NaN,"Prices, Production and Labor selected indicators",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Prices, Consumer Price Index, All items, Index","getSelectionEl(0,1,""Indicator"")","getSelectionEl(0,1,""Indicator"")",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Source: International Financial Statistics (IFS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Country,Scale,Base Year,2013,2013Q1,2013M01,2013M02,2013M03,2013Q2,...,2018M05,2018M06,2018Q3,2018M07,2018M08,2018M09,2018Q4,2018M10,2018M11,2018M12
6,NaN,"Afghanistan, Islamic Republic of",Units,2010=100,127.795,125.223,125.191,125.516,124.962,126.717,...,145.343,145.191,145.512,145.385,145.297,145.855,...,147.255,148.218,...
7,NaN,Albania,Units,2010=100,107.582,108.463,107.572,108.797,109.02,107.944,...,116.843,116.778,116.844,116.471,116.897,117.166,117.183,116.963,116.673,117.912
8,NaN,Algeria,Units,2010=100,117.522,117.774,117.691,117.514,118.116,117.346,...,149.534,151.156,148.866,148.257,148.932,149.409,...,151.193,150.518,...
9,NaN,Angola,Units,2010=100,136.131,132.31,131.299,132.379,133.254,135.115,...,329.144,332.962,342.883,337.051,341.111,350.486,...,355.287,360.209,...


The interesting data starts at index 5 in the Unnamed:1 column. The first column does not include data so it is unnecessary. The row at index 5 is the header. Also, the KPI ID is at index 1 column Unnamed:1. This information is useful.

I will import the data again and then clean the dataframe. In the code cell below I explain the purpose of every line.

In [4]:
data = pd.read_excel(files[0]) # parse excel file
kpi = data.iloc[1,1] # get the kpi name
df = data.iloc[6:, 1:] # get the data
cols = data.iloc[5,1:].to_list() # get columnnames
df.columns = cols # set columnnames
df.insert(loc=1,column="kpi", value=kpi) # add kpi at position 1
df = df.reset_index(drop=True) # reset the index and delete the old index
print2("Cleaned data v1 (one file):")
df.head()




Cleaned data v1 (one file):



,Country,kpi,Scale,Base Year,2013,2013Q1,2013M01,2013M02,2013M03,2013Q2,...,2018M05,2018M06,2018Q3,2018M07,2018M08,2018M09,2018Q4,2018M10,2018M11,2018M12
0,"Afghanistan, Islamic Republic of","Prices, Consumer Price Index, All items, Index",Units,2010=100,127.795,125.223,125.191,125.516,124.962,126.717,...,145.343,145.191,145.512,145.385,145.297,145.855,...,147.255,148.218,...
1,Albania,"Prices, Consumer Price Index, All items, Index",Units,2010=100,107.582,108.463,107.572,108.797,109.02,107.944,...,116.843,116.778,116.844,116.471,116.897,117.166,117.183,116.963,116.673,117.912
2,Algeria,"Prices, Consumer Price Index, All items, Index",Units,2010=100,117.522,117.774,117.691,117.514,118.116,117.346,...,149.534,151.156,148.866,148.257,148.932,149.409,...,151.193,150.518,...
3,Angola,"Prices, Consumer Price Index, All items, Index",Units,2010=100,136.131,132.31,131.299,132.379,133.254,135.115,...,329.144,332.962,342.883,337.051,341.111,350.486,...,355.287,360.209,...
4,Anguilla,"Prices, Consumer Price Index, All items, Index",Units,2010=100,106.355,106.065,...,...,...,106.63,...,...,...,...,...,...,...,...,...,...,...


As can be seen in the last row above, missing data is coded with three dots (...). This must be replaced with NaN. Also, in my opinion, the kpi column and all column names will look better if comma and spaces were replaced with underscores and if they were set to lowercase. Since I have added the kpi column I need to remove it again before renaming the columns. Otherwise, there will be a Value Error since there is a length mismatch between the *cols* list and the *df* columns. I reset the kpi column using the kpi variable set in the previous code cell.

In [5]:
df = df.drop(labels=["kpi"], axis=1) # drop kpi column
df.columns = [str(i).lower().replace(" ", "_") for i in cols] # clean columnnames
kpi = kpi.replace(", ", "_").replace(" ", "_").lower() # clean kpi string
df.insert(loc=1,column="kpi", value=kpi) # reset kpi column
df = df.replace("...", np.nan) # replace missing data with NaN

print2("Cleaned data v2 (one file):")
df.head()




Cleaned data v2 (one file):



,country,kpi,scale,base_year,2013,2013q1,2013m01,2013m02,2013m03,2013q2,...,2018m05,2018m06,2018q3,2018m07,2018m08,2018m09,2018q4,2018m10,2018m11,2018m12
0,"Afghanistan, Islamic Republic of",prices_consumer_price_index_all_items_index,Units,2010=100,127.795223,125.223096,125.191195,125.515928,124.962165,126.716991,...,145.342948,145.191142,145.512463,145.385089,145.297193,145.855109,NaN,147.255013,148.217621,NaN
1,Albania,prices_consumer_price_index_all_items_index,Units,2010=100,107.581663,108.463252,107.572383,108.797327,109.020045,107.943578,...,116.843122,116.777638,116.844480,116.470825,116.896941,117.165674,117.182718,116.963307,116.672751,117.912095
2,Algeria,prices_consumer_price_index_all_items_index,Units,2010=100,117.521838,117.773863,117.690670,117.514498,118.116421,117.345665,...,149.533877,151.156133,148.865889,148.256625,148.931953,149.409088,NaN,151.192836,150.517507,NaN
3,Angola,prices_consumer_price_index_all_items_index,Units,2010=100,136.131179,132.310430,131.299063,132.378692,133.253535,135.114554,...,329.144088,332.961741,342.882827,337.051456,341.111302,350.485724,NaN,355.286582,360.208527,NaN
4,Anguilla,prices_consumer_price_index_all_items_index,Units,2010=100,106.355020,106.065480,NaN,NaN,NaN,106.629711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Parse and clean all files

Now the dataframe looks pretty good. But I have only imported one excel file. Let's summarise the steps above in one cell and read in all excel files in **files**. This can be done with a for loop iterating over the list.

In [6]:
financial_data = pd.DataFrame() # initialize empty dataframe to fill
for i in files:
    data = pd.read_excel(str(i)) # parse excel file i
    kpi = data.iloc[1,1] # get kpi
    kpi = kpi.replace(", ", "_").replace(" ", "_").lower() # clean kpi and get string
    df = data.iloc[6:, 1:] # get data
    df.columns = [str(i).lower().replace(" ", "_") for i in data.iloc[5,1:]] # set columns
    df.insert(loc=1,column="kpi", value=kpi) # add information column
    df = df.replace("...", np.nan) # replace missing data with NaN
    financial_data = financial_data.append(df, ignore_index=True, sort=False) # append columns on information, period and country

print2("Cleaned data v2 (all files):")    
financial_data.head()




Cleaned data v2 (all files):



,country,kpi,scale,base_year,2013,2013q1,2013m01,2013m02,2013m03,2013q2,...,2018m08,2018m09,2018q4,2018m10,2018m11,2018m12,2013.0,2014.0,2015.0,2016.0
0,"Afghanistan, Islamic Republic of",prices_consumer_price_index_all_items_index,Units,2010=100,127.795223,125.223096,125.191195,125.515928,124.962165,126.716991,...,145.297193,145.855109,NaN,147.255013,148.217621,NaN,NaN,NaN,NaN,NaN
1,Albania,prices_consumer_price_index_all_items_index,Units,2010=100,107.581663,108.463252,107.572383,108.797327,109.020045,107.943578,...,116.896941,117.165674,117.182718,116.963307,116.672751,117.912095,NaN,NaN,NaN,NaN
2,Algeria,prices_consumer_price_index_all_items_index,Units,2010=100,117.521838,117.773863,117.690670,117.514498,118.116421,117.345665,...,148.931953,149.409088,NaN,151.192836,150.517507,NaN,NaN,NaN,NaN,NaN
3,Angola,prices_consumer_price_index_all_items_index,Units,2010=100,136.131179,132.310430,131.299063,132.378692,133.253535,135.114554,...,341.111302,350.485724,NaN,355.286582,360.208527,NaN,NaN,NaN,NaN,NaN
4,Anguilla,prices_consumer_price_index_all_items_index,Units,2010=100,106.355020,106.065480,NaN,NaN,NaN,106.629711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All excel files are now parsed and appended to the **financial_data** dataframe. But the dataframe is still not easy to analyze. Every period has its own column. Period is a variable so it should only be one column. Hence, the dataframe must be unpivoted/melted/gathered. This can be done with the pandas [melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) method. However, I have written a library called *neat_panda* which mimics the R  package *tidyr* and I think it is easier to use. But for brevity, I include both methods. I also use the *unique* method to see which periods are included.

In [7]:
cols = ["country", "kpi", "scale", "base_year"]
financial_data = gather(df=financial_data, key="period", value="value", columns=cols, invert_columns=True)

# financial_data = financial_data.melt(id_vars=id_vars, var_name="period", value_name="value")

print2("Unique periods")
financial_data.period.unique()

print2("Cleaned data v3 (all files):")
financial_data.sample(10)




Unique periods



array(['2013', '2013q1', '2013m01', '2013m02', '2013m03', '2013q2',
       '2013m04', '2013m05', '2013m06', '2013q3', '2013m07', '2013m08',
       '2013m09', '2013q4', '2013m10', '2013m11', '2013m12', '2014',
       '2014q1', '2014m01', '2014m02', '2014m03', '2014q2', '2014m04',
       '2014m05', '2014m06', '2014q3', '2014m07', '2014m08', '2014m09',
       '2014q4', '2014m10', '2014m11', '2014m12', '2015', '2015q1',
       '2015m01', '2015m02', '2015m03', '2015q2', '2015m04', '2015m05',
       '2015m06', '2015q3', '2015m07', '2015m08', '2015m09', '2015q4',
       '2015m10', '2015m11', '2015m12', '2016', '2016q1', '2016m01',
       '2016m02', '2016m03', '2016q2', '2016m04', '2016m05', '2016m06',
       '2016q3', '2016m07', '2016m08', '2016m09', '2016q4', '2016m10',
       '2016m11', '2016m12', '2017', '2017q1', '2017m01', '2017m02',
       '2017m03', '2017q2', '2017m04', '2017m05', '2017m06', '2017q3',
       '2017m07', '2017m08', '2017m09', '2017q4', '2017m10', '2017m11',
       '2017m




Cleaned data v3 (all files):



,country,kpi,scale,base_year,period,value
42428,Angola,economic_activity_industrial_production_index,Units,2010=100,2015m11,NaN
45384,New Zealand,employment_persons_number_of,Thousands,NaN,2016q1,2401.000000
40394,"Macedonia, FYR",financial_market_prices_equities_end_of_period...,Units,NaN,2015m09,74.792446
37253,Spain,prices_producer_price_index_all_commodities_index,Units,2010=100,2015q3,108.515529
77383,Philippines,financial_market_prices_equities_end_of_period...,Units,NaN,2018m03,161.007444
64991,Brunei Darussalam,unemployment_persons_number_of,Thousands,NaN,2017m05,NaN
24815,Denmark,labor_markets_unemployment_rate_percent,Units,NaN,2014m08,6.500000
11523,Poland,economic_activity_industrial_production_index,Units,2010=100,2013q4,115.676306
26159,United Kingdom,economic_activity_industrial_production_index,Units,2010=100,2014q4,99.802761
83106,Myanmar,unemployment_persons_number_of,Thousands,NaN,2018m08,NaN


The periods include quarters (e.g. 2013q1), year (e.g. 2015) and months (e.g. 2018m06). There is one problem, the last four periods in unique periods array have a dot and a zero after the year. This problem is most likely caused by a datatype error when the excel files are parsed. This is easily fixed with pandas, see below. I also set the value column to float and count the number of null values in the *value* column.

In [8]:
financial_data.period = financial_data.period.astype(str).str.replace(".0", "", regex=False) # fix problem with year
financial_data.value = financial_data.value.astype(float)

print2("Control periods:")
financial_data.period.unique()

print2("Number of nulls in the value column:")
financial_data.value.isnull().count()




Control periods:



array(['2013', '2013q1', '2013m01', '2013m02', '2013m03', '2013q2',
       '2013m04', '2013m05', '2013m06', '2013q3', '2013m07', '2013m08',
       '2013m09', '2013q4', '2013m10', '2013m11', '2013m12', '2014',
       '2014q1', '2014m01', '2014m02', '2014m03', '2014q2', '2014m04',
       '2014m05', '2014m06', '2014q3', '2014m07', '2014m08', '2014m09',
       '2014q4', '2014m10', '2014m11', '2014m12', '2015', '2015q1',
       '2015m01', '2015m02', '2015m03', '2015q2', '2015m04', '2015m05',
       '2015m06', '2015q3', '2015m07', '2015m08', '2015m09', '2015q4',
       '2015m10', '2015m11', '2015m12', '2016', '2016q1', '2016m01',
       '2016m02', '2016m03', '2016q2', '2016m04', '2016m05', '2016m06',
       '2016q3', '2016m07', '2016m08', '2016m09', '2016q4', '2016m10',
       '2016m11', '2016m12', '2017', '2017q1', '2017m01', '2017m02',
       '2017m03', '2017q2', '2017m04', '2017m05', '2017m06', '2017q3',
       '2017m07', '2017m08', '2017m09', '2017q4', '2017m10', '2017m11',
       '2017m




Number of nulls in the value column:



91160

There are 91160 rows with null values. One should always be careful when handling null values. But in this case, they are just in the way, so I will drop them and then control if there are any left.

The period column is not optimal, it would be nice if one easily could distinguish the time period. Also, it would be nice if year and period (month, quarter) were separate columns. This can be accomplished with the [apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) method and a [lambda](https://www.w3schools.com/python/python_lambda.asp) function.

I do all the above operations based on the period column. Then I replace the old period column with the new column that specifies the current month/quarter. NaN is set if the data is yearly.

In [9]:
financial_data = financial_data[financial_data.value.notnull()] # remova NaN's

columns = ["country", "kpi", "period", "value"]
financial_data = financial_data[columns] # filter out interesting columns

financial_data.insert(loc=2, column="year", value=financial_data.period.str[:4]) # insert year column

_period2 = (financial_data.period.apply(lambda x: x[-1:] if "q" in x
                                        else (x[-2:] if "m" in x
                                        else np.nan))) # create _period2 series

financial_data.insert(loc=3, column="period2", value=_period2) # insert _period2 into financial data

_time_period = financial_data.period.apply(lambda x: "quarter" if "q" in x
                                           else ("month" if "m" in x
                                           else "year")) # create _time_period series

financial_data.insert(loc=5, column="time_period", value=_time_period) # insert _time_period series into financial data

financial_data["period"] = financial_data["period2"] # overwrite old period column with period2

financial_data = financial_data.drop(labels=["period2"], axis=1) # drop period2 column

print2("Cleaned data v4 (all files):")

financial_data.sample(10)




Cleaned data v4 (all files):



,country,kpi,year,period,time_period,value
20045,Poland,prices_producer_price_index_all_commodities_index,2014,04,month,108.300000
49825,Thailand,labor_markets_unemployment_rate_percent,2016,04,month,1.040000
25655,Belarus,labor_markets_unemployment_rate_percent,2014,09,month,0.500000
68523,United Kingdom,unemployment_persons_number_of,2017,08,month,1500.673000
5115,Vietnam,labor_markets_unemployment_rate_percent,2013,2,quarter,1.944700
25082,Qatar,prices_consumer_price_index_all_items_index,2014,09,month,111.397903
58776,Brazil,economic_activity_industrial_production_index,2017,NaN,year,85.441020
53638,Iceland,economic_activity_industrial_production_index,2016,08,month,119.448972
62004,Ireland,prices_consumer_price_index_all_items_index,2017,03,month,105.105680
82540,Morocco,financial_market_prices_equities_end_of_period...,2018,07,month,92.816708


Now there are two steps left until the dataframe is tidy. First, the kpi column needs to be spread/pivoted. This because the kpi column contains multiple kpis and every kpi is a variable.  For this, I will use the spread function from *neat_panda*.

Also, the dataframe contains three different time periods (year, month, quarter). The time periods need to be split into three separate dataframes.

## 3. Merge with external data

However, it would help the analysis if the dataframe contained more information about the country, for example, region. This [link](https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv) contains a csv with countries and their region. Let's look at the data.

In [10]:
region = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")
print2("Region data:")
region.sample(10)

print2("Regions and sub-regions:")
region[["region", "sub-region"]].drop_duplicates().sort_values(by=["region","sub-region"]) # show region




Region data:



,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
44,Chile,CL,CHL,152,ISO 3166-2:CL,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0
162,Niue,NU,NIU,570,ISO 3166-2:NU,Oceania,Polynesia,NaN,9.0,61.0,NaN
46,Christmas Island,CX,CXR,162,ISO 3166-2:CX,Oceania,Australia and New Zealand,NaN,9.0,53.0,NaN
221,Timor-Leste,TL,TLS,626,ISO 3166-2:TL,Asia,South-eastern Asia,NaN,142.0,35.0,NaN
208,South Sudan,SS,SSD,728,ISO 3166-2:SS,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0
120,Kuwait,KW,KWT,414,ISO 3166-2:KW,Asia,Western Asia,NaN,142.0,145.0,NaN
88,Grenada,GD,GRD,308,ISO 3166-2:GD,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0
124,Lebanon,LB,LBN,422,ISO 3166-2:LB,Asia,Western Asia,NaN,142.0,145.0,NaN
27,"Bonaire, Sint Eustatius and Saba",BQ,BES,535,ISO 3166-2:BQ,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0
18,Bangladesh,BD,BGD,50,ISO 3166-2:BD,Asia,Southern Asia,NaN,142.0,34.0,NaN





Regions and sub-regions:



,region,sub-region
3,Africa,Northern Africa
6,Africa,Sub-Saharan Africa
7,Americas,Latin America and the Caribbean
24,Americas,Northern America
115,Asia,Central Asia
45,Asia,Eastern Asia
33,Asia,South-eastern Asia
0,Asia,Southern Asia
11,Asia,Western Asia
20,Europe,Eastern Europe


There are many columns, but I find the *name*, *region* and *sub-region* columns most interesting. The *name* column can be used as a key when joining with the *financial_data* dataframe. Optimal would have been if the original data had contained one of the other columns since it would have increased the likelihood for a perfect match. But I have to take what I have got and deal with it.

So, I filter the columns in the *region* dataframe and rename the *name* column to *country*. Then I use the [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) method to join the *financial_data* and *region* dataframes using country as key. I create a new dataframe called *financial_data2*. This is a temporary dataframe, I will use it to see which countries in *financial_data* that do not have a match in the *region* dataframe.

In [11]:
region = region[["name", "region", "sub-region"]] # filter columns
region = region.rename(columns = {"name":"country"} ) # rename column
print2("Cleaned region data:")
region.sample(10)

financial_data2 = pd.merge(left=financial_data, right=region, how="left", on="country") # join dataframes
print2("Countries with no match in region dataframe:")
pd.DataFrame(financial_data2.country[financial_data2.region.isnull()].unique()) # show countries with no region




Cleaned region data:



,country,region,sub-region
129,Lithuania,Europe,Northern Europe
244,Wallis and Futuna,Oceania,Polynesia
211,Sudan,Africa,Northern Africa
23,Benin,Africa,Sub-Saharan Africa
79,French Southern Territories,Africa,Sub-Saharan Africa
17,Bahrain,Asia,Western Asia
144,Micronesia (Federated States of),Oceania,Micronesia
120,Kuwait,Asia,Western Asia
73,Faroe Islands,Europe,Northern Europe
13,Australia,Oceania,Australia and New Zealand





Countries with no match in region dataframe:



,0
0,"Afghanistan, Islamic Republic of"
1,"Armenia, Republic of"
2,"Azerbaijan, Republic of"
3,"Bahamas, The"
4,"Bahrain, Kingdom of"
5,Bolivia
6,"China, P.R.: Hong Kong"
7,"China, P.R.: Macao"
8,"China, P.R.: Mainland"
9,"Congo, Democratic Republic of"


There are 48 *"countries"* that do not have a match. But, as can be seen above, the *"countries"* from index 36 to 46 are not countries. They are regions. These can be removed. This is done in the missing dataframe below.

Also, many of the countries in the financial_data dataframe that have no match in the region dataframe have longer names, often split by a comma, e.g. *Bahamas, The*. Therefore I create a new column in the missing dataframe called country2 where I split the country name on comma (,).

In [12]:
missing = pd.DataFrame(financial_data2.country[financial_data2.region.isnull()].unique(), columns=["country"])
missing = missing.query("index<36 | index==47").reset_index(drop=True) # remove regions

missing["country2"] = missing["country"].str.split(",", expand=True).iloc[:,0] # split on comma
print2("Countries with no match v2:")
missing




Countries with no match v2:



,country,country2
0,"Afghanistan, Islamic Republic of",Afghanistan
1,"Armenia, Republic of",Armenia
2,"Azerbaijan, Republic of",Azerbaijan
3,"Bahamas, The",Bahamas
4,"Bahrain, Kingdom of",Bahrain
5,Bolivia,Bolivia
6,"China, P.R.: Hong Kong",China
7,"China, P.R.: Macao",China
8,"China, P.R.: Mainland",China
9,"Congo, Democratic Republic of",Congo


Now I probably will get a match on for example Afghanistan, but many of the countries in the missing dataframe did not have a comma, hence I will not find a match for them. To solve this problem I will use [difflib](https://docs.python.org/3/library/difflib.html) and the method *get_close_matches*. This method returns a list of the best “good enough” matches.  A good explanation of this method can be found at [stackoverflow](https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas).

I will only use the first match from the *get_close_matches* list, i.e. the best match. Then I map this a new column, country3, in the *missing* dataframe.

In [13]:
import difflib

matches = missing["country2"].apply(lambda x: difflib.get_close_matches(x, region['country']))
matches = [i[0] if i!=[] else np.nan for i in matches]
matches

missing["country3"] = matches
print2("Countries with no match v3:")
missing

['Afghanistan',
 'Armenia',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 nan,
 'China',
 'China',
 'China',
 'Congo',
 'Congo',
 'Central African Republic',
 'Eswatini',
 'French Southern Territories',
 'Gambia',
 'Iraq',
 nan,
 'Korea, Republic of',
 'North Macedonia',
 'Indonesia',
 nan,
 'Sao Tome and Principe',
 'Serbia',
 'Sint Maarten (Dutch part)',
 'Moldova, Republic of',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 nan,
 'Timor-Leste',
 nan,
 'United States of America',
 nan,
 'Viet Nam',
 nan,
 'Yemen',
 'Netherlands']




Countries with no match v3:



,country,country2,country3
0,"Afghanistan, Islamic Republic of",Afghanistan,Afghanistan
1,"Armenia, Republic of",Armenia,Armenia
2,"Azerbaijan, Republic of",Azerbaijan,Azerbaijan
3,"Bahamas, The",Bahamas,Bahamas
4,"Bahrain, Kingdom of",Bahrain,Bahrain
5,Bolivia,Bolivia,NaN
6,"China, P.R.: Hong Kong",China,China
7,"China, P.R.: Macao",China,China
8,"China, P.R.: Mainland",China,China
9,"Congo, Democratic Republic of",Congo,Congo


Now it looks pretty good. But it is not perfect. There are some errors (e.g. is the Czech Republic not the same as the Central African Republic), I set these errors as NaN. Then I create a new dataframe called corrected which contains all corrected countries. I then overwrite the missing dataframe with the countries that do have NaN in the country3 column.

In [14]:
_index = [11, 15, 17, 18, 19, 36]

for i in _index:
    missing.iloc[i, 2] = np.nan

corrected = missing.dropna(axis=0, how="any").copy()
missing = missing[missing["country3"].isnull()].copy().reset_index(drop=True)
print2("Corrected:")
corrected
print2("Countries with no match v4:")
missing




Corrected:



,country,country2,country3
0,"Afghanistan, Islamic Republic of",Afghanistan,Afghanistan
1,"Armenia, Republic of",Armenia,Armenia
2,"Azerbaijan, Republic of",Azerbaijan,Azerbaijan
3,"Bahamas, The",Bahamas,Bahamas
4,"Bahrain, Kingdom of",Bahrain,Bahrain
6,"China, P.R.: Hong Kong",China,China
7,"China, P.R.: Macao",China,China
8,"China, P.R.: Mainland",China,China
9,"Congo, Democratic Republic of",Congo,Congo
10,"Congo, Republic of",Congo,Congo





Countries with no match v4:



,country,country2,country3
0,Bolivia,Bolivia,NaN
1,Czech Republic,Czech Republic,NaN
2,"Iran, Islamic Republic of",Iran,NaN
3,"Kosovo, Republic of",Kosovo,NaN
4,Kyrgyz Republic,Kyrgyz Republic,NaN
5,"Macedonia, FYR",Macedonia,NaN
6,"Micronesia, Federated States of",Micronesia,NaN
7,Moldova,Moldova,NaN
8,Tanzania,Tanzania,NaN
9,United Kingdom,United Kingdom,NaN


There are only 13 countries left. Below I try to find a match using the [contains](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html) method from pandas. This method works similarly to the **LIKE** keyword in SQL. I will only use the first 8 letters in the country column. Why 8? Well, because I got the best result this way :). I create a new dictionary where I set the key to the country from the country column in missing and the value to the mactched country. If there is no match, NaN is set as value.

In [15]:
mapping ={}
for i in missing["country"].values.tolist():
    _i = i[:8]
    try:
        temp = region.query("country.str.contains(@_i)", engine="python")[["country"]].iloc[0,0]
    except Exception as exc:
        temp = np.nan
    mapping[i] = temp
mapping

{'Bolivia': 'Bolivia (Plurinational State of)',
 'Czech Republic': nan,
 'Iran, Islamic Republic of': nan,
 'Kosovo, Republic of': nan,
 'Kyrgyz Republic': nan,
 'Macedonia, FYR': 'North Macedonia',
 'Micronesia, Federated States of': 'Micronesia (Federated States of)',
 'Moldova': 'Moldova, Republic of',
 'Tanzania': 'Tanzania, United Republic of',
 'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
 'Venezuela, Republica Bolivariana de': 'Venezuela (Bolivarian Republic of)',
 'West Bank and Gaza': nan,
 'Netherlands Antilles': 'Netherlands'}

Now there is only 5 countries with no match (inlcuding Netherlands Antilles, that is not situated in the Netherlands. I correct these countries manuallaly by searching for a substring in the region dataframe.

In [16]:
print2("Czech Republic")
region.query("country.str.slice(0,3) == 'Cze'")

print2("Iran")
region.query("country.str.contains('Iran')", engine="python")

print2("Kosovo")
region.query("country.str.contains('Koso')", engine="python")

print2("Kyrgyz Republic")
region.query("country.str.contains('Kyrg')", engine="python")

print2("West Bank and Gaza")
region.query("country.str.contains('Gaza')", engine="python")
region.query("country.str.contains('Bank')", engine="python")

print2("Antilles")
region.query("country.str.slice(0,3) == 'Ant'")




Czech Republic



,country,region,sub-region
59,Czechia,Europe,Eastern Europe





Iran



,country,region,sub-region
105,Iran (Islamic Republic of),Asia,Southern Asia





Kosovo



...




West Bank and Gaza



,country,region,sub-region


,country,region,sub-region





Antilles



,country,region,sub-region
8,Antarctica,NaN,NaN
9,Antigua and Barbuda,Americas,Latin America and the Caribbean


Based on the result above I manually set the value of the mapping dictionary.

In [17]:
mapping["Czech Republic"] = "Czechia"
mapping["Iran, Islamic Republic of"]= "Iran (Islamic Republic of)"
mapping["Macedonia, FYR"] = "North Macedonia"
mapping["Kyrgyz Republic"] = "Kyrgyzstan"

mapping["Netherlands Antilles"] = np.nan

mapping

{'Bolivia': 'Bolivia (Plurinational State of)',
 'Czech Republic': 'Czechia',
 'Iran, Islamic Republic of': 'Iran (Islamic Republic of)',
 'Kosovo, Republic of': nan,
 'Kyrgyz Republic': 'Kyrgyzstan',
 'Macedonia, FYR': 'North Macedonia',
 'Micronesia, Federated States of': 'Micronesia (Federated States of)',
 'Moldova': 'Moldova, Republic of',
 'Tanzania': 'Tanzania, United Republic of',
 'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
 'Venezuela, Republica Bolivariana de': 'Venezuela (Bolivarian Republic of)',
 'West Bank and Gaza': nan,
 'Netherlands Antilles': nan}

Now only Kosovo and West Bank and Gaza have missing values. I know that Kosovo is a neighbor to Serbia, and West Bank is a neighbor to Israel, so I set these countries as values. This can be done since I will use this value as a key when joining the financial_data and region dataframes.

Then I use the *mapping* dictionary to map the countries in the missing dataframe and append these to the corrected dataframe.

Then, to get correct regions in the region dataframe, I merge the corrected dataframe with the region dataframe. Since I previously dropped countries with no matching keys from the region dataframe I can append the corrected dataframe to the region dataframe.

In [18]:
region.query("country=='Israel' | country=='Serbia'").drop_duplicates()

mapping['West Bank and Gaza'] = "Israel"
mapping["Kosovo, Republic of"] = "Serbia"

missing["country3"] = missing["country"].map(mapping)

corrected = corrected.append(missing)

corrected = (pd.merge(left=corrected, right=region, left_on="country3", right_on="country")
             .drop(labels=["country_y", "country2", "country3"], axis=1)
             .rename(columns={"country_x":"country"})
            )
region = region.append(corrected)

region.drop_duplicates() # ändra denna så den kollar om det är duplicates

,country,region,sub-region
109,Israel,Asia,Western Asia
197,Serbia,Europe,Southern Europe


,country,region,sub-region
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia
5,Andorra,Europe,Southern Europe
6,Angola,Africa,Sub-Saharan Africa
7,Anguilla,Americas,Latin America and the Caribbean
8,Antarctica,NaN,NaN
9,Antigua and Barbuda,Americas,Latin America and the Caribbean


Now the region dataframe is cleaned. I join the financial dataframe with the region dataframe. Then I control that there are no missing countries. As can be seen below there are none.

In [19]:
financial_data2 = pd.merge(left=financial_data, right=region, how="left", on="country")

print2("Countries with no match in region dataframe:")
pd.DataFrame(financial_data2.country[financial_data2.region.isnull()].unique())

financial_data2.sample(10)




Countries with no match in region dataframe:



,0
0,Advanced Economies
1,CIS
2,Emerging and Developing Asia
3,Emerging and Developing Countries
4,Emerging and Developing Europe
5,Europe
6,"Middle East, North Africa, Afghanistan, and Pa..."
7,Sub-Saharan Africa
8,Western Hemisphere
9,World


,country,kpi,year,period,time_period,value,region,sub-region
53937,France,prices_consumer_price_index_all_items_index,2018,07,month,109.042918,Europe,Western Europe
27450,Nicaragua,prices_producer_price_index_all_commodities_index,2015,07,month,151.984147,Americas,Latin America and the Caribbean
43605,Brunei Darussalam,prices_consumer_price_index_all_items_index,2017,03,month,99.253150,Asia,South-eastern Asia
47236,Montenegro,economic_activity_industrial_production_index,2017,07,month,86.489667,Europe,Southern Europe
3102,Canada,financial_market_prices_equities_end_of_period...,2013,03,month,94.842605,Americas,Northern America
16401,Ireland,economic_activity_industrial_production_index,2014,3,quarter,114.257145,Europe,Northern Europe
46154,Euro Area,unemployment_persons_number_of,2017,06,month,14114.000000,NaN,NaN
375,Trinidad and Tobago,labor_force_persons_number_of,2013,NaN,year,650.150000,Americas,Latin America and the Caribbean
692,Tunisia,labor_markets_unemployment_rate_percent,2013,NaN,year,15.844975,Africa,Northern Africa
53527,United States,labor_force_persons_number_of,2018,06,month,163277.000000,Americas,Northern America


Now I need to spread the financial dataframe to get all the kpis in separate columns. Also, I only use the time period month.

In [20]:
imf = (spread(df=financial_data2, key="kpi", value="value", convert=False)
       .query("time_period=='month'")
       .drop(labels=["time_period"], axis=1)
       .reset_index(drop=True))

imf.insert(loc=3, column="year_period", value=imf.year + "-" +  imf.period)

imf.sample(10)

,country,year,period,year_period,region,sub-region,economic_activity_industrial_production_index,employment_persons_number_of,financial_market_prices_equities_end_of_period_index,labor_force_persons_number_of,labor_markets_unemployment_rate_percent,prices_consumer_price_index_all_items_index,prices_producer_price_index_all_commodities_index,unemployment_persons_number_of
12972,West Bank and Gaza,2017,04,2017-04,Asia,Western Asia,NaN,NaN,NaN,NaN,NaN,111.984373,NaN,NaN
10515,Sierra Leone,2016,12,2016-12,Africa,Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,162.664931,NaN,NaN
4100,"Eswatini, Kingdom of",2014,06,2014-06,Africa,Sub-Saharan Africa,NaN,NaN,NaN,NaN,NaN,128.459869,NaN,NaN
4305,Europe,2014,06,2014-06,nan,nan,NaN,NaN,NaN,NaN,NaN,116.180510,NaN,NaN
4540,France,2016,04,2016-04,Europe,Western Europe,102.477333,NaN,NaN,NaN,9.900000,105.674919,100.572023,2911.000
11473,Suriname,2013,04,2013-04,Americas,Latin America and the Caribbean,NaN,NaN,NaN,NaN,NaN,124.459491,NaN,NaN
214,Algeria,2013,02,2013-02,Africa,Northern Africa,NaN,NaN,NaN,NaN,NaN,117.514498,NaN,NaN
9405,Paraguay,2013,05,2013-05,Americas,Latin America and the Caribbean,123.995600,NaN,NaN,1356.143,NaN,113.665559,114.789300,NaN
11673,Switzerland,2018,09,2018-09,Europe,Western Europe,NaN,NaN,NaN,NaN,2.372137,99.394526,94.439775,106.586
2499,"China, P.R.: Macao",2017,04,2017-04,Asia,Eastern Asia,NaN,381.8,NaN,389.400,2.000000,135.269881,NaN,7.600


In [21]:
imf.shape

(13290, 14)

## 4. Save data
I choose to save data as csv file with the pandas method [to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)

In [22]:
imf.to_csv("../../imf/clean/imf_monthly.csv", index=False)

## Closing comments
Now the dataframe is tidy. In a future post, I will use R to do the same data tidying.